In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv("/Users/phupwinteaindray/Documents/ANNClassification/Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Preprocess the data
### Drop irrelevant column

In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Encode categorical variables

In [4]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

### One hot encode to geography

In [5]:
from sklearn.preprocessing import OneHotEncoder
onehot_geography = OneHotEncoder(sparse_output=False)
geo_encoder= onehot_geography.fit_transform(data[['Geography']])

In [6]:
onehot_geography.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encode_df = pd.DataFrame(geo_encoder,columns=onehot_geography.get_feature_names_out(['Geography']))

### Combine all geography encode to the original data

In [8]:
data = pd.concat([data.drop('Geography',axis=1),geo_encode_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


## Save encoder and scaler

In [9]:
with open('lable_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_geography,file)

## Divide the data into dependent and independent features

In [10]:
X = data.drop('Exited',axis=1)
y = data['Exited']

## Split the data into training and testing data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [12]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Implementation

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [15]:
## Build ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), # First hidden layer connected with input layer
    Dense(32,activation='relu'), ## Second hidden layer
    Dense(1,activation='sigmoid') # output layer

])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)

In [18]:
loss = tensorflow.keras.losses.BinaryCrossentropy()

In [19]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [20]:
log_dir = "logs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [21]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)

In [22]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights=True)

In [24]:
history = model.fit(
    X_train,y_train, validation_data = (X_test,y_test), epochs = 100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 0s 571us/step - loss: 0.3260 - accuracy: 0.8618 - val_loss: 0.3434 - val_accuracy: 0.8600
Epoch 2/100
250/250 [==============================] - 0s 520us/step - loss: 0.3225 - accuracy: 0.8686 - val_loss: 0.3428 - val_accuracy: 0.8565
Epoch 3/100
250/250 [==============================] - 0s 579us/step - loss: 0.3212 - accuracy: 0.8671 - val_loss: 0.3386 - val_accuracy: 0.8595
Epoch 4/100
250/250 [==============================] - 0s 515us/step - loss: 0.3193 - accuracy: 0.8671 - val_loss: 0.3373 - val_accuracy: 0.8640
Epoch 5/100
250/250 [==============================] - 0s 492us/step - loss: 0.3184 - accuracy: 0.8660 - val_loss: 0.3404 - val_accuracy: 0.8605
Epoch 6/100
250/250 [==============================] - 0s 499us/step - loss: 0.3147 - accuracy: 0.8705 - val_loss: 0.3365 - val_accuracy: 0.8645
Epoch 7/100
250/250 [==============================] - 0s 513us/step - loss: 0.3129 - accuracy: 0.8712 - val_loss: 0.3446 - val_ac

In [25]:
model.save('model.h5')

/Users/phupwinteaindray/Documents/ANNClassification/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
### Load Tensorboard Extension
%load_ext tensorboard

In [29]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 81986), started 0:05:34 ago. (Use '!kill 81986' to kill it.)

## Load the pickle file